# Spotify Music Library Exploration
The goal of this project was to learn working with SQL while finding some interesting insights about my Spotify music library (in this case, all my saved albums and corresponding artists).

First, we need to connect to the database to get the access to the data we want to analyze:

In [ ]:
import duckdb
con = duckdb.connect("database.db", read_only=True)

## Looking at albums and artists:

Let us take a look on the structure of the database. We have two tables containing our albums and artists with the corresponding spotify ids and other interesting data. The following two cells show the contents of these tables.

In [ ]:
# show artists database
con.sql("SELECT * FROM artists").df()

In [ ]:
# show albums database
con.sql("SELECT * FROM albums").df()

The albums and artist have a many-to-many relationship between each other. Artist can publish many albums and album can be recorded by multiple artists. Each of the album-artist connections is present in the album_artist table, which can be used to connect the artist and albums table to reveal some more interesting information. The following cell shows how to list the artists in an alphabetical order together with the list of their albums.

In [ ]:
# show list of artists in alphabetical order and the list of their albums
con.sql("""
        SELECT artists.name AS Artist, LIST(albums.name) as Albums
        FROM artists INNER JOIN album_artist ON artists.spotify_id=album_artist.artist_id INNER JOIN albums ON album_artist.album_id=albums.spotify_id
        GROUP BY artists.name
        ORDER BY artists.name ASC
    """).df()

One can use this functionality to find all the albums from a given artist (next cell) or even all other albums by the artist of a given album (the cell after that).

In [ ]:
# find all albums by the given artist
given_artist = "Sufjan Stevens"
con.execute("""
        SELECT DISTINCT artists.name AS artist_name, albums.name AS album_name  
        FROM albums 
        LEFT JOIN album_artist ON albums.spotify_id=album_artist.album_id
        JOIN artists ON album_artist.artist_id=artists.spotify_id
        WHERE artist_name = ?
        """, (given_artist,)).df()

In [ ]:
# find artist based on an album and then other albums by that artist
analyzed_album = "softscars"
artist_id = con.execute("""        
        SELECT DISTINCT artists.spotify_id AS artist_id, artists.name AS artist_name, albums.name AS album_name  
        FROM albums 
        LEFT JOIN album_artist ON albums.spotify_id=album_artist.album_id
        JOIN artists ON album_artist.artist_id=artists.spotify_id
        WHERE album_name=?;
        """,(analyzed_album,)).df()["artist_id"][0]
con.execute("""
        SELECT DISTINCT artists.name AS artist_name, albums.name AS album_name  
        FROM albums 
        LEFT JOIN album_artist ON albums.spotify_id=album_artist.album_id
        JOIN artists ON album_artist.artist_id=artists.spotify_id
        WHERE artists.spotify_id=? AND albums.name!=?;
        """,(artist_id, analyzed_album,)).df()

We can even use this database to be a bit nostalgic and give us tips on what to listen to. The following cell will randomly pick and show us an album from the given year.

In [ ]:
# show random album from a given year
year = 2023
con.sql(f"""
        SELECT artists.name as artist_name, albums.name as album_name, total_tracks, release_date, added_at, popularity 
        FROM albums 
        JOIN album_artist ON albums.spotify_id = album_artist.album_id
        JOIN artists ON artists.spotify_id = album_artist.artist_id 
        WHERE added_at BETWEEN '{year}-01-01 00:00:00.000' AND '{year}-12-31 23:59:59.999' 
        ORDER BY RANDOM()  
        LIMIT 1  
        """).df()

## Looking at genres and tags:

To make the analysis more interesting, all the genres for each of the artists pulled from spotify were saved in another table. That way, we can see what are the genre trends in our Spotify library. For example, the next cell shows the top 10 genres with a list of corresponding artists.

In [ ]:
# show 10 most frequent genres
con.sql("""
        SELECT genre, COUNT(name) AS count, LIST(name) AS artists 
        FROM artists INNER JOIN artist_genre ON artists.spotify_id=artist_genre.artist_id 
        GROUP BY genre ORDER BY count DESC LIMIT 10
    """).df()

We also obtained the MusicBrainz id for as many artists we could and used those to obtain the LastFM tags of each of the artist for even more insight into the data.

In [ ]:
# show 
con.sql("""
        SELECT tag, SUM(weight) AS summed_weight, COUNT(name) AS count, LIST(name) AS aritists
        FROM artists INNER JOIN artist_tag ON artists.mbid=artist_tag.mbid
        GROUP BY tag ORDER BY summed_weight DESC LIMIT 10
        """).df()

Having these, we can do all kinds of stuff like trying to find similar artists based on shared genres or tags as shown in the following two cells.

In [ ]:
# based on artist, find other artists based on the genre and their albums
analyzed_artist = "AURORA"
genres_list = con.execute("""
        SELECT genre FROM artists 
        JOIN artist_genre ON artists.spotify_id = artist_genre.artist_id
        WHERE artists.name = ?
        """, (analyzed_artist,)).df()["genre"].to_list()
placeholders = ", ".join("?" for _ in genres_list) # create the appropriate number of ? -> ?, ?, ?, ?, ...
unknowns = genres_list + [analyzed_artist]
con.execute(f"""
        SELECT artists.name, LIST(genre) AS genres, COUNT(genre) AS same_genre_count        
        FROM artists 
        JOIN artist_genre ON artists.spotify_id = artist_genre.artist_id
        WHERE genre IN ({placeholders}) AND artists.name != ?
        GROUP BY artists.name
        ORDER BY same_genre_count DESC
        """, unknowns).df()

In [ ]:
# based on artist, find other artists based on the genre and their albums
analyzed_artist = "AURORA"
threshold = 30
tags_list = con.execute(f"""
        SELECT tag FROM artists 
        JOIN artist_tag ON artists.mbid = artist_tag.mbid
        WHERE artists.name = ? AND weight > {threshold}
        """, (analyzed_artist,)).df()["tag"].to_list()
placeholders = ", ".join("?" for _ in tags_list) # create the appropriate number of ? -> ?, ?, ?, ?, ...
unknowns = tags_list + [analyzed_artist]
con.execute(f"""
        SELECT artists.name, LIST(tag) AS tags, SUM(weight) AS same_tag_count        
        FROM artists 
        JOIN artist_tag ON artists.mbid = artist_tag.mbid
        WHERE tag IN ({placeholders}) AND artists.name != ? AND weight > {threshold}
        GROUP BY artists.name
        ORDER BY same_tag_count DESC
        """, unknowns).df()